# AI Trader - Backtesting Tutorial

This notebook demonstrates how to use the AI Trader backtesting framework to:
1. Fetch historical market data
2. Test trading strategies
3. Analyze portfolio performance
4. Compare different strategy configurations

## Prerequisites

Make sure you've installed all dependencies:
```bash
pip install -r requirements.txt
```

In [ ]:
# Setup
import sys
sys.path.append('..')

from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt

from src.api.data_api import DataAPI
from src.api.strategy_api import StrategyAPI
from src.api.backtest_api import BacktestAPI
from src.strategy.ma_crossover import MACrossoverStrategy

# Display settings
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)

## 1. Data Layer - Fetching Historical Data

The `DataAPI` provides a user-friendly interface for fetching and caching market data.

In [ ]:
# Initialize DataAPI
data_api = DataAPI()

# Define symbols and date range
symbols = ['AAPL', 'MSFT', 'GOOGL']
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')

print(f"Fetching data for {symbols}")
print(f"Period: {start_date} to {end_date}")

In [ ]:
# Fetch data for Apple
aapl_data = data_api.get_daily_bars('AAPL', start_date, end_date)

print(f"AAPL data shape: {aapl_data.shape}")
print(f"\nFirst 5 rows:")
aapl_data.head()

In [ ]:
# Plot closing prices
plt.figure(figsize=(14, 6))
plt.plot(aapl_data.index, aapl_data['close'], label='AAPL Close', linewidth=2)
plt.title('AAPL Closing Price (Last 12 Months)', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price ($)', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 2. Strategy Layer - Testing Trading Strategies

The `StrategyAPI` allows you to test individual strategies on single symbols.

In [ ]:
# Initialize StrategyAPI
strategy_api = StrategyAPI(data_api=data_api)

# Create MA Crossover strategy
strategy = MACrossoverStrategy({
    'fast_period': 50,
    'slow_period': 200
})

print(f"Strategy: {strategy.__class__.__name__}")
print(f"Parameters: {strategy.params}")

In [ ]:
# Generate signals
signals = strategy_api.get_signals('AAPL', strategy, start_date, end_date)

print(f"Generated {len(signals)} signals")
print(f"Buy signals: {(signals == 1.0).sum()}")
print(f"Sell signals: {(signals == -1.0).sum()}")
print(f"\nSignal dates:")
signals[signals != 0]

In [ ]:
# Get strategy data with indicators
strategy_data = strategy_api.get_strategy_data('AAPL', strategy, start_date, end_date)

print(f"Strategy data shape: {strategy_data.shape}")
print(f"\nColumns: {list(strategy_data.columns)}")
print(f"\nLast 5 rows:")
strategy_data[['close', 'fast_ma', 'slow_ma', 'signal']].tail()

In [ ]:
# Visualize strategy signals
plt.figure(figsize=(14, 8))

# Plot price and moving averages
plt.plot(strategy_data.index, strategy_data['close'], label='Close Price', linewidth=2, alpha=0.7)
plt.plot(strategy_data.index, strategy_data['fast_ma'], label=f'Fast MA ({strategy.params["fast_period"]})', linewidth=1.5)
plt.plot(strategy_data.index, strategy_data['slow_ma'], label=f'Slow MA ({strategy.params["slow_period"]})', linewidth=1.5)

# Mark buy/sell signals
buy_signals = strategy_data[strategy_data['signal'] == 1.0]
sell_signals = strategy_data[strategy_data['signal'] == -1.0]

plt.scatter(buy_signals.index, buy_signals['close'], 
           color='green', marker='^', s=200, label='Buy Signal', zorder=5)
plt.scatter(sell_signals.index, sell_signals['close'], 
           color='red', marker='v', s=200, label='Sell Signal', zorder=5)

plt.title(f'AAPL - MA Crossover Strategy ({strategy.params["fast_period"]}/{strategy.params["slow_period"]})', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price ($)', fontsize=12)
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Run simple backtest (single symbol)
results = strategy_api.backtest('AAPL', strategy, start_date, end_date, initial_capital=10000)

print("=" * 70)
print("SINGLE SYMBOL BACKTEST RESULTS")
print("=" * 70)
print(f"Initial Capital:    ${results['initial_capital']:,.2f}")
print(f"Final Value:        ${results['final_value']:,.2f}")
print(f"\nStrategy Return:    {results['total_return']:.2%}")
print(f"Buy & Hold Return:  {results['buy_and_hold_return']:.2%}")
print(f"\nAlpha:              {results['total_return'] - results['buy_and_hold_return']:.2%}")
print(f"\nTotal Trades:       {results['num_trades']}")
print(f"Buy Signals:        {results['num_buy_signals']}")
print(f"Sell Signals:       {results['num_sell_signals']}")
print(f"Round Trips:        {results['num_round_trips']}")
print("=" * 70)

## 3. Portfolio Backtesting - Multi-Symbol Portfolio

The `BacktestAPI` provides comprehensive backtesting with:
- Portfolio allocation across multiple symbols
- Risk management
- Rebalancing
- Performance metrics

In [ ]:
# Initialize BacktestAPI
backtest_api = BacktestAPI()

# Run backtest on portfolio
print("Running portfolio backtest...")
portfolio_result = backtest_api.run_ma_crossover(
    symbols=symbols,
    start_date=start_date,
    end_date=end_date,
    fast_period=50,
    slow_period=200,
    initial_cash=100000.0,
    rebalance_frequency='weekly',
    max_positions=10
)

print("✓ Backtest complete")

In [ ]:
# Display formatted results
print(backtest_api.format_results(portfolio_result))

In [ ]:
# Get equity curve
equity_curve = backtest_api.get_equity_curve(portfolio_result)

print(f"Equity curve shape: {equity_curve.shape}")
print(f"\nLast 5 days:")
equity_curve[['portfolio_value', 'cash', 'positions_value']].tail()

In [ ]:
# Plot equity curve
plt.figure(figsize=(14, 8))

plt.subplot(2, 1, 1)
plt.plot(equity_curve.index, equity_curve['portfolio_value'], linewidth=2, label='Portfolio Value')
plt.axhline(y=portfolio_result.initial_value, color='gray', linestyle='--', alpha=0.5, label='Initial Value')
plt.title('Portfolio Value Over Time', fontsize=14)
plt.ylabel('Value ($)', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
plt.plot(equity_curve.index, equity_curve['cash'], label='Cash', linewidth=2, color='green')
plt.plot(equity_curve.index, equity_curve['positions_value'], label='Positions Value', linewidth=2, color='blue')
plt.title('Cash vs Positions Value', fontsize=14)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Value ($)', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Get trade history
trades = backtest_api.get_trades(portfolio_result)

if not trades.empty:
    print(f"Total trades: {len(trades)}")
    print(f"\nFirst 10 trades:")
    print(trades.head(10).to_string())
else:
    print("No trades executed")

In [ ]:
# Get daily returns
daily_returns = backtest_api.get_daily_returns(portfolio_result)

print(f"Daily returns shape: {daily_returns.shape}")
print(f"\nStatistics:")
print(f"Mean:     {daily_returns.mean():.4f}")
print(f"Std Dev:  {daily_returns.std():.4f}")
print(f"Min:      {daily_returns.min():.4f}")
print(f"Max:      {daily_returns.max():.4f}")

In [ ]:
# Plot daily returns distribution
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.plot(daily_returns.index, daily_returns.values, linewidth=1, alpha=0.7)
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
plt.title('Daily Returns Over Time', fontsize=14)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Return', fontsize=12)
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(daily_returns.values, bins=50, edgecolor='black', alpha=0.7)
plt.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Zero Return')
plt.axvline(x=daily_returns.mean(), color='green', linestyle='--', linewidth=2, label='Mean Return')
plt.title('Daily Returns Distribution', fontsize=14)
plt.xlabel('Return', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Strategy Comparison

Compare multiple strategy configurations to find the best parameters.

In [ ]:
# Define configurations to test
configs = {
    'MA(10/30)': {'fast_period': 10, 'slow_period': 30},
    'MA(20/50)': {'fast_period': 20, 'slow_period': 50},
    'MA(50/200)': {'fast_period': 50, 'slow_period': 200},
}

results_comparison = {}

print("Running strategy comparison...\n")
for name, params in configs.items():
    print(f"Testing {name}...")
    result = backtest_api.run_ma_crossover(
        symbols=symbols,
        start_date=start_date,
        end_date=end_date,
        fast_period=params['fast_period'],
        slow_period=params['slow_period'],
        initial_cash=100000.0,
        rebalance_frequency='weekly'
    )
    results_comparison[name] = result
    print(f"  Return: {result.total_return_pct:.2f}%")

print("\n✓ Comparison complete")

In [ ]:
# Display comparison table
print(backtest_api.format_comparison(results_comparison))

In [ ]:
# Get comparison DataFrame
comparison_df = backtest_api.compare_results(results_comparison)
comparison_df

In [ ]:
# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Total Return
axes[0, 0].bar(comparison_df.index, comparison_df['Total Return (%)'])
axes[0, 0].set_title('Total Return (%)', fontsize=12)
axes[0, 0].set_ylabel('%', fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

# Max Drawdown
axes[0, 1].bar(comparison_df.index, comparison_df['Max Drawdown (%)'], color='red', alpha=0.7)
axes[0, 1].set_title('Max Drawdown (%)', fontsize=12)
axes[0, 1].set_ylabel('%', fontsize=10)
axes[0, 1].grid(True, alpha=0.3)

# Sharpe Ratio
axes[1, 0].bar(comparison_df.index, comparison_df['Sharpe Ratio'], color='green', alpha=0.7)
axes[1, 0].set_title('Sharpe Ratio', fontsize=12)
axes[1, 0].set_ylabel('Ratio', fontsize=10)
axes[1, 0].grid(True, alpha=0.3)

# Number of Trades
axes[1, 1].bar(comparison_df.index, comparison_df['Num Trades'], color='purple', alpha=0.7)
axes[1, 1].set_title('Number of Trades', fontsize=12)
axes[1, 1].set_ylabel('Count', fontsize=10)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Compare equity curves
plt.figure(figsize=(14, 8))

for name, result in results_comparison.items():
    equity = backtest_api.get_equity_curve(result)
    plt.plot(equity.index, equity['portfolio_value'], label=name, linewidth=2)

plt.title('Strategy Comparison - Equity Curves', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Portfolio Value ($)', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Key Takeaways

This tutorial demonstrated:
1. ✅ **Data Layer**: Fetching and caching historical data with `DataAPI`
2. ✅ **Strategy Layer**: Testing strategies with `StrategyAPI`
3. ✅ **Portfolio Backtesting**: Multi-symbol backtests with `BacktestAPI`
4. ✅ **Performance Analysis**: Equity curves, returns, and metrics
5. ✅ **Strategy Comparison**: Testing multiple configurations

## Next Steps

- Experiment with different strategy parameters
- Test on different symbol portfolios
- Try weekly or monthly rebalancing
- Adjust risk parameters (max position size, cash reserve)
- Implement custom strategies by extending the `Strategy` base class